<a href="https://colab.research.google.com/github/specM7/DSGP_Group_33_Brain_Tumor_Predictor/blob/Meningioma_Adrian_2425482/Meningioma_Adrian_2425482.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI-Powered Brain Tumor Predictor for Meningioma(Adrian)


## Project Goal
Build a Convolutional Neural Network (CNN) to classify brain MRI images into 4 tumor categories.

## Technologies Used
- **TensorFlow/Keras** - Deep Learning Framework
- **OpenCV** - Image Processing
- **NumPy** - Numerical Computing


## Model Specifications
- **Input**: 224x224 grayscale MRI images
- **Architecture**: Custom CNN with 3 convolutional blocks
- **Output**: 4-class classification (tumor types)
- **Training**: 25 epochs with validation split

###  Objectives
1. Build a robust CNN model for brain tumor classification
2. Achieve >90% accuracy on test data
3. Provide interpretable results with visualization

In [ ]:
import os, glob, cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


##  Extract Dataset
Unzips your file and shows folder structure.

---

## 2️ Dataset Preparation

###  Upload Dataset
Upload your `DataSet.zip` file containing:
```
DataSet/
├── Training/
│   ├── glioma/
│   ├── meningioma/
│   ├── notumor/
│   └── pituitary/
└── Testing/
    ├── glioma/
    ├── meningioma/
    ├── notumor/
    └── pituitary/
```

**Click the upload button below to select your dataset ZIP file.**

In [ ]:
from google.colab import files

uploaded = files.upload()        # PICK the real DataSet.zip from your PC

zip_name = list(uploaded.keys())[0]
print("Uploaded:", zip_name)

# Unzip into /content/DataSet_raw
!rm -rf /content/DataSet_raw
!mkdir -p /content/DataSet_raw
!unzip -q "$zip_name" -d /content/DataSet_raw

print("\nTop level under /content/DataSet_raw:")
!ls /content/DataSet_raw

print("\nAny 'Training' folders:")
!find /content/DataSet_raw -maxdepth 5 -type d -iname "Training"


Saving DataSet.zip to DataSet.zip
Uploaded: DataSet.zip

Top level under /content/DataSet_raw:
DataSet

Any 'Training' folders:
/content/DataSet_raw/DataSet/Training


##  Set Paths
Look at previous output, find your Training folder path, and paste it in next cell.

Example: `/content/DataSet_raw/DataSet/Training`

In [1]:

training_path = "/content/DataSet_raw/DataSet/Training"   # example, edit to your real one

raw_root = os.path.dirname(training_path)
print("training_path:", training_path)
print("raw_root:", raw_root)
print("RAW training folders:", os.listdir(training_path))

img_size = (224, 224)
batch_size = 32

train_dir = training_path
test_dir  = os.path.join(raw_root, "Testing")
print("train_dir:", train_dir)
print("test_dir:", test_dir)

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=42,
    color_mode="grayscale"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=42,
    color_mode="grayscale"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    color_mode="grayscale"
)

print("Classes:", train_ds.class_names)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.9, 1.1],
    horizontal_flip=True,
    validation_split=0.2
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    subset='training',
    seed=42
)

val_ds = train_datagen.flow_from_directory(           # ← note: we still augment lightly on val
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    subset='validation',
    seed=42
)

test_ds = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    shuffle=False,
    seed=42
)

print("Classes:", train_ds.class_indices)


NameError: name 'os' is not defined

## 6️ Model Evaluation

###  Testing on Unseen Data

Now we'll evaluate the model on completely unseen test data to measure real-world performance.

**Test Set:** Independent images never seen during training

In [ ]:
model = models.Sequential([
    layers.Input(shape=(*img_size, 1)),
    layers.Rescaling(1./255),

    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.Recall(class_id=1, name='meningioma_recall'),
        tf.keras.metrics.Precision(class_id=1, name='meningioma_precision')
    ]
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25
)

test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,168,900 (42.61 MB)

 Trainable params: 11,168,900 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 476s 3s/step - accuracy: 0.4857 - loss: 1.0995 - val_accuracy: 0.8039 - val_loss: 0.5963
Epoch 2/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 486s 3s/step - accuracy: 0.7435 - loss: 0.6368 - val_accuracy: 0.8634 - val_loss: 0.4425
Epoch 3/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 505s 3s/step - accuracy: 0.8174 - loss: 0.4823 - val_accuracy: 0.8573 - val_loss: 0.3899
Epoch 4/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 496s 3s/step - accuracy: 0.8515 - loss: 0.4212 - val_accuracy: 0.8774 - val_loss: 0.3646
Epoch 5/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 497s 3s/step - accuracy: 0.8635 - loss: 0.3843 - val_accuracy: 0.8914 - val_loss: 0.3240
Epoch 6/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 501s 3s/step - accuracy: 0.8866 - loss: 0.3293 - val_accuracy: 0.8923 - val_loss: 0.3100
Epoch 7/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 509s 3s/step - accuracy: 0.8801 - loss: 0.3219 - val_accuracy: 0.8993 - val_loss: 0.2939
Epoch 8/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 484s 3s/step - accuracy: 0.8964 - loss: 0.2778 - val_accu

##  Save & Export Model

###  Saving Trained Model

The model will be saved in HDF5 format (.h5) which includes:
- ✅ Model architecture
- ✅ Trained weights
- ✅ Optimizer state
- ✅ Training configuration

**File:** `brain_tumor_mri_model.h5`

In [ ]:
model.save("brain_tumor_mri_model.h5")


In [ ]:
from google.colab import files
files.download("brain_tumor_mri_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>